In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import shapefile
import gdal
import osgeo
import json
import json 
import pylab as pl
import scipy.optimize as opt
import itertools
import os, fnmatch

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from matplotlib.ticker import NullFormatter

from sklearn.cluster import KMeans
from sklearn import preprocessing, metrics, model_selection
from sklearn.metrics import jaccard_similarity_score, f1_score, log_loss, confusion_matrix

import pylab as pl
import scipy.optimize as opt

import folium

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


# Clustering and Segmenting the Neighborhoods of New York:  A study in characteristics of a 'Hot Restaurant' area

### In a prior notebook, we have imported and analyzed extensive data from the United States Census Bureau regarding the rate of change of selected financial, academic and sociocultural characteristics of the households residing in census tracts on the island of Manhattan.  The results of that work are to be found in the data files in this repository.

## In this notebook, we will analyze those results, combine them with other data from publicly available services, such as Foursquare, and attempt to characterize through cluster analysis the combination of characteristics common to census tracts that might be supportive of rapid development of restaurants that would appeal and be successful in attracting the affluent professional clientele that is of interest to our client.

###  We begin by gathering the results of the census data studies into a single 'feature' dataframe suitable for cluster analysis.  The data has been extensively cleaned, but on a feature by feature basis, so will require additional normalization prior to the analysis.  The analysis will make use of the KMeans module avalable from SciKit-Learn (sklearn), will be optimized through use of the 'elbow method' and will be analyzed statistically for significance.  We will provide visualization of the results of that work.

####  As further background, the object of the census data analysis was to develop a measure of the 'momentum' of each feature measured as the rate and direction of change of the value of the feature over a six-year series of five-year estimations, one each concluding in the years 2012 - 2017 (the most recent data available from the USCB).  This was done on a census tract by census tract basis to the extent that data was available. There are roughly 280 census tracts covering Manhattan; in most cases data for at least 80% of thos census tracts was available for every, or nearly every, year.  We first import the momentum files and then will merge them into a single dataframe for cluster analysis.

In [2]:
mom_list = ['HHNPINC_k', 'HHPINC_k', 'HHPHD_k', 'HHPRD_k', 'HHMSTR_k', 'HHNLA_k', 'HHLA_k', 'RENTS_k']

In [3]:

pds={}

for i in range(len(mom_list)):
    tr = '/users/richardkornblith/Data_Science/NYCHR/Data_for_NYCHR/'
    full_path = tr+mom_list[i]+'.csv'
    df_t = pd.read_csv(full_path)
    pds[mom_list[i]]=df_t
    
pds.keys()


dict_keys(['HHNPINC_k', 'HHPINC_k', 'HHPHD_k', 'HHPRD_k', 'HHMSTR_k', 'HHNLA_k', 'HHLA_k', 'RENTS_k'])

In [4]:
hhnpinc_df = pds['HHNPINC_k']
hhprd_df = pds['HHPRD_k']
hhmstr_df = pds['HHMSTR_k']
hhnla_df = pds['HHNLA_k']
hhla_df = pds['HHLA_k']
rents_df = pds['RENTS_k']
hhpinc_df = pds['HHPINC_k']
hhphd_df = pds['HHPHD_k']

In [5]:
x=hhnpinc_df.merge(hhpinc_df,how='outer', on='Tracts')
y=x.merge(hhphd_df, how='outer', on='Tracts')
z=y.merge(hhmstr_df, how='outer', on='Tracts')
xx=z.merge(hhnla_df, how='outer', on='Tracts')
yy=xx.merge(hhla_df, how='outer', on='Tracts')
zz= yy.merge(rents_df, how='outer', on='Tracts')
features = zz.merge(hhprd_df, how='outer', on='Tracts')


In [6]:
print(features.info())
features.columns


<class 'pandas.core.frame.DataFrame'>
Int64Index: 283 entries, 0 to 282
Data columns (total 21 columns):
Unnamed: 0_x    280 non-null float64
0_x             280 non-null float64
Tracts          283 non-null int64
Unnamed: 0_y    280 non-null float64
0_y             280 non-null float64
1               280 non-null float64
2               280 non-null float64
3               280 non-null float64
4               280 non-null float64
Unnamed: 0_x    282 non-null float64
0_x             282 non-null float64
Unnamed: 0_y    282 non-null float64
0_y             282 non-null float64
Unnamed: 0_x    280 non-null float64
0_x             280 non-null float64
Unnamed: 0_y    280 non-null float64
0_y             280 non-null float64
Unnamed: 0_x    250 non-null float64
0_x             250 non-null float64
Unnamed: 0_y    282 non-null float64
0_y             282 non-null float64
dtypes: float64(20), int64(1)
memory usage: 48.6 KB
None


Index(['Unnamed: 0_x', '0_x', 'Tracts', 'Unnamed: 0_y', '0_y', '1', '2', '3',
       '4', 'Unnamed: 0_x', '0_x', 'Unnamed: 0_y', '0_y', 'Unnamed: 0_x',
       '0_x', 'Unnamed: 0_y', '0_y', 'Unnamed: 0_x', '0_x', 'Unnamed: 0_y',
       '0_y'],
      dtype='object')

In [8]:
 features.drop(labels = ['Unnamed: 0_x','Unnamed: 0_y','1', '2', '3','4'], axis=1,inplace=True)

In [9]:
features.head(30)

,0_x,Tracts,0_y,0_x,0_y,0_x,0_y,0_x,0_y
0,-0.016268,36061000201,0.016268,-0.000446,-0.001631,-0.000156,-0.000156,0.000000,-0.001631
1,0.013393,36061000202,-0.013393,-0.000330,-0.000125,0.006933,0.006933,0.000000,-0.000125
2,-0.004739,36061000600,0.004739,-0.001548,0.000182,0.010362,0.010362,0.000000,0.000182
3,-0.000953,36061000700,0.000953,-0.005116,0.009302,0.020193,0.020193,0.127787,0.009302
4,-0.012722,36061000800,0.012722,0.002825,-0.000589,0.003300,0.003300,0.003077,-0.000589
5,-0.011629,36061000900,0.011629,-0.001467,0.054165,0.005969,0.005969,0.037785,0.054165
6,-0.011445,36061001001,0.011445,0.002385,0.000411,0.006624,0.006624,0.000000,0.000411
7,-0.004408,36061001002,0.004408,0.000000,-0.000433,-0.005407,-0.005407,0.000000,-0.000433
8,-0.014580,36061001200,0.014580,0.001097,0.000609,0.004751,0.004751,NaN,0.000609
9,-0.003055,36061001300,0.003055,0.001535,0.016564,0.023960,0.023960,0.070768,0.016564
